In [1]:
import pandas as pd

# 1. 데이터 로드
try:
    df = pd.read_csv('XEM7360.csv')
except FileNotFoundError:
    print("XEM7360.csv 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
    df = pd.DataFrame(columns=['Connector', 'Pin', 'FPGA Pin', 'Pin Attributes', 'FPGA Clock In', 'FPGA I/O', 'I/O Bank'])

# ---------------------------------------------------------
# 2. 설정 및 신호 정의
# ---------------------------------------------------------

what_mc = 'MC1'  # 사용할 커넥터

# [수정됨] 클럭 신호 수동 매핑 (신호명 : 핀번호)
# ★★★ 여기에 원하는 핀 번호를 직접 입력하세요 ★★★
manual_clock_map = {
    "clk_clock_generator": 7,   # 예: MC2-98에 할당
}

# 일반 데이터/제어 신호 정의
regular_signals = []
regular_signals.append("margin_pin_mc1[0]")

for i in range(66):
    regular_signals.append(f"input_streaming_data_from_fpga_to_asic[{i}]")
regular_signals.append("input_streaming_valid_from_fpga_to_asic")
regular_signals.append("input_streaming_ready_from_asic_to_fpga")
regular_signals.append("start_training_signal_from_fpga_to_asic")
regular_signals.append("start_ready_from_asic_to_fpga")
regular_signals.append("start_inference_signal_from_fpga_to_asic")
regular_signals.append("inferenced_label_from_asic_to_fpga")
regular_signals.append("reset_n_from_fpga_to_asic")

# for i in range(10): 
#     regular_signals.append(f"margin_pin[{i}]")


# ---------------------------------------------------------
# 3. 데이터 전처리 및 필터링
# ---------------------------------------------------------

df['FPGA Clock In'] = df['FPGA Clock In'].fillna(False)
df['FPGA I/O'] = df['FPGA I/O'].fillna(False)
df['I/O Bank'] = df['I/O Bank'].fillna(0).astype(int)

# 유효한 핀 필터링 (일반 신호 할당용 후보군)
valid_pins = df[
    (df['Connector'] == what_mc) & 
    (df['I/O Bank'].isin([12, 15, 16, 32])) &
    (df['FPGA I/O'].astype(str).str.upper() == 'TRUE')
].copy()

# 일반 신호는 핀 번호 역순으로 할당 (MC2-100 -> MC2-99 ...)
valid_pins = valid_pins.sort_values(by='Pin', ascending=False)


# ---------------------------------------------------------
# 4. 핀 할당 로직
# ---------------------------------------------------------

used_pin_indices = [] 
xdc_output = []

print("#--- Generating XDC Constraints ---\n")

# --- [Step 1] 클럭 신호 수동 할당 ---
for signal, pin_num in manual_clock_map.items():
    # 해당 커넥터와 핀 번호에 맞는 행 찾기
    target_pin = df[(df['Connector'] == what_mc) & (df['Pin'] == pin_num)]
    
    if target_pin.empty:
        print(f"# Error: Pin {what_mc}-{pin_num} not found in CSV for signal {signal}")
        continue
    
    # 정보 추출
    pin_row = target_pin.iloc[0]
    
    # 사용된 핀으로 등록 (나중에 중복 할당 방지)
    used_pin_indices.append(pin_row.name)
    
    # XDC 생성
    xdc_output.append(f"# {what_mc}-{pin_row['Pin']} (Manually Assigned Clock)")
    xdc_output.append(f"set_property PACKAGE_PIN {pin_row['FPGA Pin']} [get_ports {{{signal}}}]")
    xdc_output.append(f"set_property IOSTANDARD LVCMOS18 [get_ports {{{signal}}}]")


# --- [Step 2] 일반 신호 할당 ---

# 1. 이미 수동으로 할당한 핀(클럭 등)은 후보에서 제외
remaining_pins = valid_pins.drop(used_pin_indices, errors='ignore')

# (참고) 남은 핀 중 클럭 가능 핀을 제외하는 코드는 주석 처리 유지 (요청사항 반영)
# remaining_pins = remaining_pins[
#     remaining_pins['FPGA Clock In'].astype(str).str.upper() != 'TRUE'
# ]

for signal in regular_signals:
    if remaining_pins.empty:
        print(f"# Error: Not enough regular pins for {signal}!")
        break
        
    pin_row = remaining_pins.iloc[0]
    
    # 선택된 핀 제거 (다음 루프를 위해)
    remaining_pins = remaining_pins.iloc[1:]
    
    xdc_output.append(f"# {what_mc}-{pin_row['Pin']}")
    xdc_output.append(f"set_property PACKAGE_PIN {pin_row['FPGA Pin']} [get_ports {{{signal}}}]")
    xdc_output.append(f"set_property IOSTANDARD LVCMOS18 [get_ports {{{signal}}}]")

# ---------------------------------------------------------
# 5. 결과 출력
# ---------------------------------------------------------
for line in xdc_output:
    print(line)


print('\n\n\n\n\n\n\n\n\n\n\n\n')

#--- Generating XDC Constraints ---

# MC1-7 (Manually Assigned Clock)
set_property PACKAGE_PIN G17 [get_ports {clk_clock_generator}]
set_property IOSTANDARD LVCMOS18 [get_ports {clk_clock_generator}]
# MC1-118
set_property PACKAGE_PIN A8 [get_ports {margin_pin_mc1[0]}]
set_property IOSTANDARD LVCMOS18 [get_ports {margin_pin_mc1[0]}]
# MC1-117
set_property PACKAGE_PIN H8 [get_ports {input_streaming_data_from_fpga_to_asic[0]}]
set_property IOSTANDARD LVCMOS18 [get_ports {input_streaming_data_from_fpga_to_asic[0]}]
# MC1-116
set_property PACKAGE_PIN A9 [get_ports {input_streaming_data_from_fpga_to_asic[1]}]
set_property IOSTANDARD LVCMOS18 [get_ports {input_streaming_data_from_fpga_to_asic[1]}]
# MC1-115
set_property PACKAGE_PIN H9 [get_ports {input_streaming_data_from_fpga_to_asic[2]}]
set_property IOSTANDARD LVCMOS18 [get_ports {input_streaming_data_from_fpga_to_asic[2]}]
# MC1-112
set_property PACKAGE_PIN B9 [get_ports {input_streaming_data_from_fpga_to_asic[3]}]
set_property IOSTANDA

# MC2 에 마진 핀 할당

In [2]:
import pandas as pd

# 1. 데이터 로드
try:
    df = pd.read_csv('XEM7360.csv')
except FileNotFoundError:
    print("XEM7360.csv 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
    df = pd.DataFrame(columns=['Connector', 'Pin', 'FPGA Pin', 'Pin Attributes', 'FPGA Clock In', 'FPGA I/O', 'I/O Bank'])

# ---------------------------------------------------------
# 2. 설정 및 신호 정의
# ---------------------------------------------------------

what_mc = 'MC2'  # 사용할 커넥터

# [수정됨] 클럭 신호 수동 매핑 (신호명 : 핀번호)
# ★★★ 여기에 원하는 핀 번호를 직접 입력하세요 ★★★
manual_clock_map = {
}

# 일반 데이터/제어 신호 정의
regular_signals = []

for i in range(22, -1, -1):
    regular_signals.append(f"margin_pin_mc2[{i}]")

# for i in range(10): 
#     regular_signals.append(f"margin_pin[{i}]")


# ---------------------------------------------------------
# 3. 데이터 전처리 및 필터링
# ---------------------------------------------------------

df['FPGA Clock In'] = df['FPGA Clock In'].fillna(False)
df['FPGA I/O'] = df['FPGA I/O'].fillna(False)
df['I/O Bank'] = df['I/O Bank'].fillna(0).astype(int)

# 유효한 핀 필터링 (일반 신호 할당용 후보군)
valid_pins = df[
    (df['Connector'] == what_mc) & 
    (df['I/O Bank'].isin([12, 15, 16, 32])) &
    (df['FPGA I/O'].astype(str).str.upper() == 'TRUE') &
    (df['Pin']%2 == 1)
].copy()

# 일반 신호는 핀 번호 역순으로 할당 (MC2-100 -> MC2-99 ...)
valid_pins = valid_pins.sort_values(by='Pin', ascending=False)


# ---------------------------------------------------------
# 4. 핀 할당 로직
# ---------------------------------------------------------

used_pin_indices = [] 
xdc_output = []

print("#--- Generating XDC Constraints ---\n")

# --- [Step 1] 클럭 신호 수동 할당 ---
for signal, pin_num in manual_clock_map.items():
    # 해당 커넥터와 핀 번호에 맞는 행 찾기
    target_pin = df[(df['Connector'] == what_mc) & (df['Pin'] == pin_num)]
    
    if target_pin.empty:
        print(f"# Error: Pin {what_mc}-{pin_num} not found in CSV for signal {signal}")
        continue
    
    # 정보 추출
    pin_row = target_pin.iloc[0]
    
    # 사용된 핀으로 등록 (나중에 중복 할당 방지)
    used_pin_indices.append(pin_row.name)
    
    # XDC 생성
    xdc_output.append(f"# {what_mc}-{pin_row['Pin']} (Manually Assigned Clock)")
    xdc_output.append(f"set_property PACKAGE_PIN {pin_row['FPGA Pin']} [get_ports {{{signal}}}]")
    xdc_output.append(f"set_property IOSTANDARD LVCMOS18 [get_ports {{{signal}}}]")


# --- [Step 2] 일반 신호 할당 ---

# 1. 이미 수동으로 할당한 핀(클럭 등)은 후보에서 제외
remaining_pins = valid_pins.drop(used_pin_indices, errors='ignore')

# (참고) 남은 핀 중 클럭 가능 핀을 제외하는 코드는 주석 처리 유지 (요청사항 반영)
# remaining_pins = remaining_pins[
#     remaining_pins['FPGA Clock In'].astype(str).str.upper() != 'TRUE'
# ]

for signal in regular_signals:
    if remaining_pins.empty:
        print(f"# Error: Not enough regular pins for {signal}!")
        break
        
    pin_row = remaining_pins.iloc[0]
    
    # 선택된 핀 제거 (다음 루프를 위해)
    remaining_pins = remaining_pins.iloc[1:]
    
    xdc_output.append(f"# {what_mc}-{pin_row['Pin']}")
    xdc_output.append(f"set_property PACKAGE_PIN {pin_row['FPGA Pin']} [get_ports {{{signal}}}]")
    xdc_output.append(f"set_property IOSTANDARD LVCMOS18 [get_ports {{{signal}}}]")

# ---------------------------------------------------------
# 5. 결과 출력
# ---------------------------------------------------------
for line in xdc_output:
    print(line)

print('\n\n\n\n\n\n\n\n\n\n\n\n')


#--- Generating XDC Constraints ---

# MC2-117
set_property PACKAGE_PIN AE15 [get_ports {margin_pin_mc2[22]}]
set_property IOSTANDARD LVCMOS18 [get_ports {margin_pin_mc2[22]}]
# MC2-115
set_property PACKAGE_PIN AD15 [get_ports {margin_pin_mc2[21]}]
set_property IOSTANDARD LVCMOS18 [get_ports {margin_pin_mc2[21]}]
# MC2-111
set_property PACKAGE_PIN AF15 [get_ports {margin_pin_mc2[20]}]
set_property IOSTANDARD LVCMOS18 [get_ports {margin_pin_mc2[20]}]
# MC2-109
set_property PACKAGE_PIN AF14 [get_ports {margin_pin_mc2[19]}]
set_property IOSTANDARD LVCMOS18 [get_ports {margin_pin_mc2[19]}]
# MC2-107
set_property PACKAGE_PIN V14 [get_ports {margin_pin_mc2[18]}]
set_property IOSTANDARD LVCMOS18 [get_ports {margin_pin_mc2[18]}]
# MC2-105
set_property PACKAGE_PIN AB15 [get_ports {margin_pin_mc2[17]}]
set_property IOSTANDARD LVCMOS18 [get_ports {margin_pin_mc2[17]}]
# MC2-103
set_property PACKAGE_PIN AB14 [get_ports {margin_pin_mc2[16]}]
set_property IOSTANDARD LVCMOS18 [get_ports {margin_pin_m